# Training Models

# Exercises

## 1. 

Stochastic GD and Mini-batch GD are probably the most appropriate options for datasets with too many features. Batch GD can also be a good option if out-of-core training is not strictly necessary (i.e., the training set fits in memory).

## 2.
Different scales can affect the convergence speed of Gradient Descent, since the shape of the cost function would be much like an elongated bowl. Regularized models can also be affected and may converge to a suboptimal solution: since regularization penalizes large wights, features with smaller values might be ignored compared to features with larger values.

## 3.
No, because the cost function of a Logistic Regression model is convex.

## 4.
No, because Stochastic GD and Mini-batch GD will never really converge. However, for convex problems and a learning rate not too high, all Gradient Descent algorithms will approach the global optimum, yielding very similar models (and even then the models are different still).

## 5.
If only the validation error going up, it is likely that the model is overfitting the training data, and the training should be stopped. If both errors are going up, the model is diverging due to a learning rate too high.

## 6.
No, because Mini-batch Gradient Descent is not guaranteed to make progress at every single training step. Due to the nature of the algorithm, the cost function can bounce around a little, going up and down, and still be improving on average. If the model has not improved for a long time, however, it is more likely that it has already peaked and training should be stopped.

## 7.
Stochastic GD has the fastest training time, so it will *probably* be the first to reach the vicinity of the global optimum. However, only Batch Gradient Descent will actually converge, while both Stochastic GD and Mini-batch GD will bounce around the optimum (unless the training rate is gradually reduced during training).

## 8.
The large gap probably indicates that the model is not generalizing very well for some reason, or even overfitting the training data. Some approaches to solve this are:
- Maybe the model is too complex (i.e., has too many degrees of freedom), so reducing the polynomial degree might help;
- Regularization of the model is also a viable option to reduce the complexity of the model;
- Increasing the size of the training set may help with the generalization issues.

## 9.
In this situation, the model is likely suffering from high bias, due to the assumption that the data is less complex than it intrinsically is, which leads to underfitting. The regularization parameter $\alpha$ should be reduced to help with this.

## 10.

1. Generally, a model with some regularization tends to perform better than a model without any. Ridge Regression is a good default and should be preferred over plain Linear Regression.
2. Lasso encourages sparse models by means of the $\ell_1$ penalty. This can help with automatic feature selection (if we have a reason to suspect that only a few of them matter) and even noise filtering. If we have no reason to believe this, Ridge Regression should be preferred.
3. Elastic Net offers a good compromise between both regularization strategies. This is especially appealing since Lasso may behave erratically in some cases (when several features are strongly correlated or when there are more features than training instances). The main disadvantage is that it add an additional hyperparameter to tune. For Lasso without the erratic behavior, Elastic Net can be used with an `l1_ratio` close to 1.

## 11.
Two Logistic Regression classifiers would be more appropriate, since these classes are not exclusive (i.e., all four combinations are possible).

## 12.
This exercise requires us to implement Batch Gradient Descent (with early stopping) for Softmax Regression, without using scikit-learn, but I decided to use some helper functions in order to easily demonstrate the implementation working.

We need to implement the cost function and gradient equations. The cross entropy cost function is given by

$$
J(\rm{\Theta}) = -\frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}y_{k}^{(i)}\log\left(\hat{p}_{k}^{(i)}\right)
$$

and the cross entropy gradient vector for class $k$ is given by

$$
\nabla_{\rm{\theta}^{(k)}}J(\rm{\Theta}) = \frac{1}{m}\sum_{i=1}^{m}\left(\hat{p}_{k}^{(i)} - \hat{y}_{k}^{(i)} \right)x^{(i)}
$$

In [1]:
"""
Batch Gradient Descent with early stopping for Softmax Regression (aka Multinomial Logistic Regression)
"""

import numpy as np


def softmax(logits):
    exps = np.exp(logits)
    return exps / np.sum(exps, axis=1, keepdims=True)


def softmax_regression_fit(X_train, y_train, X_val, y_val, penalty=None, alpha=0.1, max_iter=5001, lr=0.01, verbose=False):
    """
    Fit a Softmax Regression model with early stopping and optional regularization
    
    Parameters
    ----------
    X_train : array-like of shape (n_samples, n_features)
        Training data
    y_train : array-like of shape (n_samples, n_classes)
        One-hot-encoded training labels
    X_val : array-like of shape (n_samples, n_features)
        Validation data
    y_val : array-like of shape (n_samples, n_classes)
        One-hot-encoded validation labels
    penalty : {'l1', 'l2', None}, default=None
        Regularization method
    alpha : float, default=0.1
        Regularization hyperparameter
    max_iter : int, default=1000
        Maximum number of iterations
    lr : float, default=0.01
        Learning rate
    verbose : bool, default=False
        Be verbose
        
    Returns
    -------
    Theta : array-like of shape (n_features + 1, n_classes)
        Model parameter matrix
    """
    # Add the bias term for every instance
    X_bias = np.c_[np.ones(len(X_train)), X_train]
    X_bias_val = np.c_[np.ones(len(X_val)), X_val]
    
    # Number of instances
    m = X_bias.shape[0]
    # Number of features + bias term
    n_inputs = X_bias.shape[1] 
    # Number of classes
    n_outputs = y_train.shape[1]
    
    # Add tiny value to compute the log of the probability of instance i belonging to class x,
    # in order to avoid nan values when this probability is 0
    epsilon = 1e-9
    
    # Randomly initialize parameter matrix Theta. Each column corresponds to the parameter vector of a given class
    Theta = np.random.randn(n_inputs, n_outputs)
    
    best_loss = np.infty
    
    if penalty not in ['l1', 'l2']:
        alpha = 0
    
    for iteration in range(max_iter):
        # Compute class scores
        logits = X_bias.dot(Theta)
        # Estimate the probabilites of each instance belonging to each class 
        proba = softmax(logits)
        # Compute the loss function
        loss = -np.mean(np.sum(y_train * np.log(proba + epsilon), axis=1))
        # Add the regularization term
        if penalty == 'l1':
            loss += alpha * np.sum(np.abs(Theta[1:]))
        elif penalty == 'l2':
            loss += alpha * 0.5 * np.sum(np.square(Theta[1:]))
        # Compute the gradient vectors
        gradients = 1/m * X_bias.T.dot(proba - y_train) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
        # Update parameter matrix
        Theta -= lr * gradients
        
        # Measure the loss on the validation set to verify if we should stop early
        logits = X_bias_val.dot(Theta)
        proba = softmax(logits)
        loss = -np.mean(np.sum(y_val * np.log(proba + epsilon), axis=1))
        if penalty == 'l1':
            loss += alpha * np.sum(np.abs(Theta[1:]))
        elif penalty == 'l2':
            loss += alpha * 0.5 * np.sum(np.square(Theta[1:]))
            
        if verbose and iteration % 1000 == 0:
            print(f"Loss at iteration {iteration}: {loss:.4f}")
        
        if loss < best_loss: 
            best_loss = loss
        else:
            if verbose: print(f"Early stopping at iteration {iteration} with loss {loss}")
            break
    
    return Theta


def softmax_regression_predict(Theta, X):
    """
    Softmax Regression of the data X with parameters Theta
    
    Parameters
    ----------
    Theta : array-like of shape (n_features + 1, n_classes)
        Model parameter matrix
    X : array-like of shape (n_samples, n_features)
        Input data
        
    Returns
    -------
    y_pred : array-like of shape (n_samples, n_classes)
        Predicted labels of X
    """
    X_bias = np.c_[np.ones(len(X)), X]
    
    logits = X_bias.dot(Theta)
    proba = softmax(logits)
    
    return np.argmax(proba, axis=1)

Now that we've implemented it, we need to test it. We will use the iris dataset for that, with only two features. Let's load it and split into the train, validation and test sets:

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

X, y = load_iris(return_X_y=True)
# Use only petal length and width
X = X[:, (2, 3)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

y_train = OneHotEncoder(sparse=False).fit_transform(y_train[:, np.newaxis])
y_val = OneHotEncoder(sparse=False).fit_transform(y_val[:, np.newaxis])
y_test = OneHotEncoder(sparse=False).fit_transform(y_test[:, np.newaxis])

We can now train the model and retrieve its parameters:

In [3]:
Theta = softmax_regression_fit(X_train, y_train, X_val, y_val, penalty='l2', lr=0.15, alpha=0.1, max_iter=50000, verbose=True)

Loss at iteration 0: 4.6653
Loss at iteration 1000: 0.5456
Loss at iteration 2000: 0.5364
Loss at iteration 3000: 0.5348
Loss at iteration 4000: 0.5345
Loss at iteration 5000: 0.5343
Loss at iteration 6000: 0.5343
Loss at iteration 7000: 0.5343
Loss at iteration 8000: 0.5343
Loss at iteration 9000: 0.5343
Loss at iteration 10000: 0.5343
Loss at iteration 11000: 0.5343
Loss at iteration 12000: 0.5343
Loss at iteration 13000: 0.5343
Loss at iteration 14000: 0.5343
Loss at iteration 15000: 0.5343
Loss at iteration 16000: 0.5343
Loss at iteration 17000: 0.5343
Loss at iteration 18000: 0.5343
Loss at iteration 19000: 0.5343
Loss at iteration 20000: 0.5343
Early stopping at iteration 20444 with loss 0.5343070427809481


Looks good, it even stopped earlier! Let's take a look at how it performs on the validation data:

In [4]:
y_true = np.argmax(y_val, axis=1)
y_pred = softmax_regression_predict(Theta, X_val)
np.mean(y_pred == y_true)

0.9166666666666666

Over 90% accuracy! How about the test data? Let's see:

In [5]:
y_true = np.argmax(y_test, axis=1)
y_pred = softmax_regression_predict(Theta, X_test)
np.mean(y_pred == y_true)

1.0

Awesome, it performed even better on the test set! 